# Configurando o Ambiente

In [1]:
pip install requests pandas

In [2]:
import requests
import pandas as pd

# Respondendo as Perguntas

## Pergunta 1: Quantos feriados há no Brasil em todo o ano de 2024?

In [3]:
# Feriados no Brasil em 2024
url_feriados = "https://date.nager.at/api/v3/PublicHolidays/2024/BR"
res = requests.get(url_feriados)
feriados = pd.DataFrame(res.json())

# Resultado
print(f"Total de feriados no Brasil em 2024: {len(feriados)}")

Total de feriados no Brasil em 2024: 15


## Pergunta 2: Qual mês de 2024 tem o maior número de feriados?

In [4]:
# Converter coluna de data
feriados["date"] = pd.to_datetime(feriados["date"])
feriados["mes"] = feriados["date"].dt.month

# Contar feriados por mês
feriados_por_mes = feriados["mes"].value_counts().sort_values(ascending=False)
mes_mais_feriados = feriados_por_mes.idxmax()

print(f"Mês com mais feriados: {mes_mais_feriados} ({feriados_por_mes.max()} feriados)")
feriados_por_mes.sort_index()

Mês com mais feriados: 11 (3 feriados)


,count
mes,
1,1
2,2
3,2
4,1
5,2
7,1
9,1
10,1
11,3


## Pergunta 3: Quantos feriados em 2024 caem em dias úteis (segunda a sexta)?

In [5]:
feriados["dia_semana"] = feriados["date"].dt.weekday  # 0 = segunda, ..., 6 = domingo

dias_uteis = feriados[feriados["dia_semana"] < 5]
print(f"Feriados em dias úteis: {len(dias_uteis)}")

Feriados em dias úteis: 10


##  Pergunta 4: Qual foi a temperatura média em cada mês de 01/01/2024 a 01/08/2024 no Rio de Janeiro?

In [6]:
url_clima = (
    "https://archive-api.open-meteo.com/v1/era5"
    "?latitude=-22.91&longitude=-43.17"
    "&start_date=2024-01-01&end_date=2024-08-01"
    "&daily=temperature_2m_max,temperature_2m_min"
    "&timezone=America%2FSao_Paulo"
)

res_clima = requests.get(url_clima)
weather_data = pd.DataFrame(res_clima.json()['daily'])
weather_data['time'] = pd.to_datetime(weather_data['time'])
weather_data["mes"] = weather_data["time"].dt.month
weather_data["temp_media"] = (weather_data["temperature_2m_max"] + weather_data["temperature_2m_min"]) / 2

media_por_mes = weather_data.groupby("mes")["temp_media"].mean().round(2)
print("Temperatura média por mês (°C):")
print(media_por_mes)


Temperatura média por mês (°C):
mes
1    27.16
2    27.77
3    26.94
4    25.70
5    25.64
6    23.28
7    21.73
8    21.80
Name: temp_media, dtype: float64


## Pergunta 5: Qual foi o tempo predominante em cada mês nesse período?

In [7]:
# Incluindo weather_code
url_clima_com_weather = (
    "https://archive-api.open-meteo.com/v1/era5"
    "?latitude=-22.91&longitude=-43.17"
    "&start_date=2024-01-01&end_date=2024-08-01"
    "&daily=weathercode"
    "&timezone=America%2FSao_Paulo"
)

res_weather = requests.get(url_clima_com_weather)
weather_codes = pd.DataFrame(res_weather.json()['daily'])
weather_codes["time"] = pd.to_datetime(weather_codes["time"])
weather_codes["mes"] = weather_codes["time"].dt.month

# Código de tempo predominante (mais frequente) por mês
predominio_tempo = weather_codes.groupby("mes")["weathercode"].agg(lambda x: x.mode().iloc[0])
print("Tempo predominante por mês (weather_code):")
print(predominio_tempo)


Tempo predominante por mês (weather_code):
mes
1    63
2    51
3    51
4     3
5    51
6     3
7     3
8     3
Name: weathercode, dtype: int64


##6. Qual foi o tempo e a temperatura média em cada feriado entre 01/01/2024 e 01/08/2024?

In [8]:
# Merge entre feriados e dados climáticos
weather_data_full = weather_data.merge(weather_codes[['time', 'weathercode']], on='time')
feriados_ate_agosto = feriados[feriados["date"] <= "2024-08-01"]

feriados_clima = feriados_ate_agosto.merge(weather_data_full, left_on='date', right_on='time', how='left')
feriados_clima_resultado = feriados_clima[["localName", "date", "temp_media", "weathercode"]]

feriados_clima_resultado


,localName,date,temp_media,weathercode
0,Confraternização Universal,2024-01-01,24.95,51
1,Carnaval,2024-02-12,30.85,3
2,Carnaval,2024-02-13,31.15,51
3,Sexta-feira Santa,2024-03-29,25.75,61
4,Domingo de Páscoa,2024-03-31,24.75,53
5,Dia de Tiradentes,2024-04-21,23.55,2
6,Dia do Trabalhador,2024-05-01,28.35,0
7,Corpus Christi,2024-05-30,20.95,51
8,Revolução Constitucionalista de 1932,2024-07-09,22.05,53


## 7. Houve algum feriado “não aproveitável” em 2024?

In [14]:
# Requisição do clima
url_clima = (
    "https://archive-api.open-meteo.com/v1/era5"
    "?latitude=-22.91&longitude=-43.17"
    "&start_date=2024-01-01&end_date=2024-08-01"
    "&daily=temperature_2m_max,temperature_2m_min,weathercode"
    "&timezone=America%2FSao_Paulo"
)
clima = pd.DataFrame(requests.get(url_clima).json()["daily"])
clima["time"] = pd.to_datetime(clima["time"])
clima["temp_media"] = (clima["temperature_2m_max"] + clima["temperature_2m_min"]) / 2

# Merge clima + feriados
clima_feriados = feriados_ate_agosto.merge(clima, left_on="date", right_on="time", how="left")

# Função para definir se foi não aproveitável
def nao_aproveitavel(temp, code, nome_feriado):
    if "Carnaval" in nome_feriado:
        return False  # Frio ou calor, com sol de rachar ou chuva forte, o carioca sempre aproveita o carnaval
    if pd.isna(temp) or pd.isna(code):
        return True
    if temp < 20:
        return True
    if code in [3, 45, 48, 51, 53, 55, 56, 57, 61, 63, 65, 66, 67,
                71, 73, 75, 77, 80, 81, 82, 85, 86, 95, 96, 99]:
        return True
    return False

# Aplicar a função
clima_feriados["nao_aproveitavel"] = clima_feriados.apply(
    lambda row: nao_aproveitavel(row["temp_media"], row["weathercode"], row["localName"]), axis=1
)

# Filtrar feriados não aproveitáveis
feriados_nao_aproveitaveis = clima_feriados[clima_feriados["nao_aproveitavel"]][
    ["localName", "date", "temp_media", "weathercode"]
]

# Mostrar resultado
feriados_nao_aproveitaveis

,localName,date,temp_media,weathercode
0,Confraternização Universal,2024-01-01,24.95,51
3,Sexta-feira Santa,2024-03-29,25.75,61
4,Domingo de Páscoa,2024-03-31,24.75,53
7,Corpus Christi,2024-05-30,20.95,51
8,Revolução Constitucionalista de 1932,2024-07-09,22.05,53


#8. Qual foi o feriado “mais aproveitável”?

In [10]:
feriados_clima_resultado["aproveitavel"] = ~feriados_clima_resultado["nao_aproveitavel"]

feriado_top = feriados_clima_resultado[feriados_clima_resultado["aproveitavel"]].sort_values(
    by="temp_media", ascending=False
).head(1)

feriado_top[["localName", "date", "temp_media", "weathercode"]]


<ipython-input-10-2e2b64e6385d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feriados_clima_resultado["aproveitavel"] = ~feriados_clima_resultado["nao_aproveitavel"]


,localName,date,temp_media,weathercode
6,Dia do Trabalhador,2024-05-01,28.35,0
